# K Nearest Neighbor Evaluation

In [1]:
import os
import numpy as np
import torch

from sklearn.neighbors import KNeighborsClassifier

from bring_features_and_file_paths import get_BirdNET_detections
from bring_features_and_file_paths import get_spectrogram_time_mark_in_file
from bring_features_and_file_paths import bring_features_and_file_paths
from bring_features_and_file_paths import from_spectrogram_path_to_BirdNET_output_path

import matplotlib.pyplot as plt

from tqdm import tqdm

import seaborn as sns

In [2]:
features_path = '/projects/MultiActiveAI/Samir_VICReg_Outputs/features_third_pass'
# clusters_path = '/projects/MultiActiveAI/Samir_VICReg_Outputs/clusters_third_pass'
# include_indices_path = '/projects/MultiActiveAI/Samir_VICReg_Outputs/indices_third_pass/include_indices.pth'
# label_predictions_path = '/projects/MultiActiveAI/Samir_VICReg_Outputs/indices_third_pass/label_predictions.pth'
# indices_path = '/projects/MultiActiveAI/Samir_VICReg_Outputs/indices_third_pass/indices.pth'

In [3]:
feats, fpaths = bring_features_and_file_paths(features_path, sub_sample=None)
# feats, fpaths = bring_features_and_file_paths('/grand/projects/BirdAudio/Soundscapes/Second_Features/', sub_sample=None)
indices = torch.randperm(feats.size()[0])
feats=feats[indices]
fpaths=fpaths[indices]

We have 2136286 feature vectors.


In [4]:
aspect='detection'

vectors=[]
labels=[]
BirdNET_BASE_PATH = '/grand/projects/BirdAudio/Soundscapes/BirdNET_Output'
Spectrograms_BASE_PATH = '/grand/projects/BirdAudio/Soundscapes/Second_Filtered_Spectrograms/'
# Spectrograms_BASE_PATH = '/grand/projects/BirdAudio/Soundscapes/First_Filtered_Spectrograms'
spectrogram_duration = 9.0
for k, (fpath, feat) in enumerate(tqdm(zip(fpaths,feats), total=len(feats))):
    file_path = ''.join([chr(int(x)) for x in fpath]).replace('~','')
    BirdNET_PATH = from_spectrogram_path_to_BirdNET_output_path(fpath)

    try:
        start_time = get_spectrogram_time_mark_in_file(fpath, spectrogram_duration)
        
        interval = (start_time, start_time + spectrogram_duration)
        data_elements = get_BirdNET_detections(os.path.join(BirdNET_BASE_PATH, BirdNET_PATH), interval, confidence_threshold = 0.0)
        for data_element in data_elements:
            vectors.append(np.array(feat))
            labels.append(data_element[aspect])

    except:
        print("Something was wrong with")
        print(file_path)


    if k > 200000:
        break

vectors=np.array(vectors)
labels=np.array(labels)
unique_labels=np.unique(labels)
aux=[]
for label in labels:
    aux.append(np.where(unique_labels==label)[0].item())
    
labels=np.array(aux)

  1%|          | 22558/2136286 [01:36<1:18:15, 450.16it/s]

Something was wrong with
                                                                                 


  1%|▏         | 28966/2136286 [01:51<1:17:00, 456.13it/s]

Something was wrong with
                                                                                 


  2%|▏         | 40034/2136286 [02:16<1:18:56, 442.54it/s]

...

  8%|▊         | 179454/2136286 [07:31<1:14:33, 437.44it/s]

Something was wrong with
                                                                                 


  9%|▉         | 187429/2136286 [07:50<1:12:00, 451.07it/s]

Something was wrong with
                                                                                 


  9%|▉         | 200001/2136286 [08:18<1:20:26, 401.19it/s]


In [5]:
if aspect=='detection':
    mask=np.array([unique_labels=='Blue Jay'])
    mask=np.logical_or(mask,np.array([unique_labels=='Eastern Wood-Pewee']))
#     mask=np.logical_or(mask,np.array([unique_labels=='Great Crested Flycatcher']))
#     mask=np.logical_or(mask,np.array([unique_labels=='Hairy Woodpecker']))
    mask=np.logical_or(mask,np.array([unique_labels=='Indigo Bunting']))
#     mask=np.logical_or(mask,np.array([unique_labels=='Ovenbird']))
#     mask=np.logical_or(mask,np.array([unique_labels=='Red-eyed Vireo']))
#     mask=np.logical_or(mask,np.array([unique_labels=='Scarlet Tanager']))
#     mask=np.logical_or(mask,np.array([unique_labels=='Tree Swallow']))
    mask=np.logical_or(mask,np.array([unique_labels=='No detection']))

    chosen_labels=np.where(mask[0])[0]
    chosen_labels

    def find_indices(A, B):
        indices = np.where(np.isin(A, B))
        return indices

    indices=find_indices(labels, chosen_labels)[0]

    vectors = vectors[indices]
    labels = labels[indices]


In [6]:
vectors.shape

(179985, 2048)

In [7]:
labels.shape

(179985,)

In [8]:
unique_labels

array(['American Crow', 'American Goldfinch', 'American Redstart',
       'American Robin', 'Baltimore Oriole', 'Barn Swallow',
       'Bay-breasted Warbler', 'Belted Kingfisher',
       'Black-and-white Warbler', 'Black-capped Chickadee',
       'Black-crowned Night-Heron', 'Black-throated Green Warbler',
       'Blackburnian Warbler', 'Blackpoll Warbler', 'Blue Jay',
       'Blue-gray Gnatcatcher', 'Blue-headed Vireo', 'Brown Thrasher',
       'Brown-headed Cowbird', 'Canada Goose', 'Caspian Tern',
       'Cedar Waxwing', 'Chestnut-sided Warbler', 'Chimney Swift',
       'Chipping Sparrow', 'Common Grackle', 'Common Nighthawk',
       'Common Yellowthroat', "Cooper's Hawk", 'Dark-eyed Junco',
       'Double-crested Cormorant', 'Downy Woodpecker', 'Eastern Bluebird',
       'Eastern Kingbird', 'Eastern Phoebe', 'Eastern Towhee',
       'Eastern Wood-Pewee', 'European Starling', 'Field Sparrow',
       'Golden-winged Warbler', 'Gray Catbird', 'Gray-cheeked Thrush',
       'Great Blue H

In [ ]:
performances=[]
performances_per_label=[]
jumps=list(np.arange(1,21,1))
for n in tqdm(jumps):
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(vectors, labels)
    label_predictions=neigh.predict(vectors)
    performances.append(np.sum(label_predictions==labels)/labels.shape[0])
    
    per_label_performance=[]
    for label in range(len(unique_labels)):
        indices=np.where(labels==label)
        per_label_performance.append(np.sum(label_predictions[indices]==label)/indices[0].shape[0])
        
    per_label_performance=np.array(per_label_performance)
    performances_per_label.append(per_label_performance)

performances=np.array(performances)
performances_per_label=np.array(performances_per_label)

  0%|          | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_1872853/2637576098.py:13: RuntimeWarning: invalid value encountered in long_scalars
  per_label_performance.append(np.sum(label_predictions[indices]==label)/indices[0].shape[0])
  5%|▌         | 1/20 [14:25<4:34:09, 865.75s/it]/tmp/ipykernel_1872853/2637576098.py:13: RuntimeWarning: invalid value encountered in long_scalars
  per_label_performance.append(np.sum(label_predictions[indices]==label)/indices[0].shape[0])
 10%|█         | 2/20 [29:25<4:25:38, 885.46s/it]/tmp/ipykernel_1872853/2637576098.py:13: RuntimeWarning: invalid value encountered in long_scalars
  per_label_performance.append(np.sum(label_predictions[indices]==label)/indices[0].shape[0])
 15%|█▌        | 3/20 [46:05<4:25:44, 937.88s/it]/tmp/ipykernel_1872853/2637576098.py:13: RuntimeWarning: invalid value encountered in long_scalars
  per_label_performance.append(np.sum(label_predictions[indices]==label)/indices[0].shape[0])
 20%|██        | 4/20 [1:04:10<4:25:33, 995

In [ ]:
#aux=np.array([int(j+1) for j in range(jumps)])
aux=jumps
plt.figure(figsize=(15,8))
plt.plot(aux, performances, marker='o')
plt.xticks(aux)
plt.xlabel('Number of neighbors')
plt.ylabel('Performance')

In [ ]:
# sns.set(font_scale=20)
sns.set (rc = {'figure.figsize':(5,10)})
#sns.set (rc = {'figure.figsize':(55, 10)})
sns.heatmap(performances_per_label[:,chosen_labels], xticklabels=unique_labels[chosen_labels], yticklabels=aux, annot=True, fmt=".1f")
# sns.heatmap(performances_per_label, xticklabels=unique_labels, yticklabels=aux, annot=True, fmt=".1f")
#plt.xlabel('Devices')
plt.xlabel('Birds')
plt.ylabel('Number of neighbors')

In [ ]:
unique_labels[chosen_labels]

ls: cannot access '/lcrc': No such file or directory
